In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import time
import os
from datetime import datetime

from model_final_rnn import Model
from configs_final_rnn import config
from tensorflow.core.protobuf import rewriter_config_pb2
from sklearn.model_selection import train_test_split

In [2]:
def data_loader_ss_subject(time_bin,sub_id):
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,0:time_bin]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,0:time_bin]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')

    X_train, X_val, idx_train, idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.1, random_state=21)

    y_train = y_train_valid[idx_train]
    person_train = person_train_valid[idx_train]
    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]
    
    idx11 = np.arange(0,X_test.shape[2],2,dtype=int)
    idx21 = np.arange(1,X_test.shape[2],2,dtype=int)
    X_test_ss1 = np.take(X_test,idx11,axis=2)
    X_test_ss2 = np.take(X_test,idx21,axis=2)
    X_test = np.concatenate((X_test_ss1,X_test_ss2),axis=0)
    y_test = np.concatenate((y_test,y_test),axis=0)
    person_test = np.concatenate((person_test,person_test),axis=0)
    
    idx12 = np.arange(0,X_train.shape[2],2,dtype=int)
    idx22 = np.arange(1,X_train.shape[2],2,dtype=int)
    X_train_ss1 = np.take(X_train,idx12,axis=2)
    X_train_ss2 = np.take(X_train,idx22,axis=2)
    X_train = np.concatenate((X_train_ss1,X_train_ss2),axis=0)
    y_train = np.concatenate((y_train,y_train),axis=0)
    person_train = np.concatenate((person_train,person_train),axis=0)
    
    idx13 = np.arange(0,X_val.shape[2],2,dtype=int)
    idx23 = np.arange(1,X_val.shape[2],2,dtype=int)
    X_val_ss1 = np.take(X_val,idx13,axis=2)
    X_val_ss2 = np.take(X_val,idx23,axis=2)
    X_val = np.concatenate((X_val_ss1,X_val_ss2),axis=0)
    y_val = np.concatenate((y_val,y_val),axis=0)
    person_val = np.concatenate((person_val,person_val),axis=0)

    tl = X_train.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train = np.reshape(X_train,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))
    
    if sub_id<9:
        subid_test = np.where(person_test==sub_id)
        person_test = np.take(person_test,subid_test[0],axis=0)
        X_test = np.take(X_test,subid_test[0],axis=0)
        y_test = np.take(y_test,subid_test[0],axis=0)
        subid_val = np.where(person_val==sub_id)
        person_val = np.take(person_val,subid_val[0],axis=0)
        X_val = np.take(X_val,subid_val[0],axis=0)
        y_val = np.take(y_val,subid_val[0],axis=0)
        subid_train = np.where(person_train==sub_id)
        person_train = np.take(person_train,subid_train[0],axis=0)
        X_train = np.take(X_train,subid_train[0],axis=0)
        y_train = np.take(y_train,subid_train[0],axis=0)

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m2 = (y_train==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_train,m2,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train, 'labels': y_train, 'person': person_train}
    test_set = {'data': X_test, 'labels': y_test, 'person': person_test}
    val_set = {'data': X_val, 'labels': y_val, 'person': person_val}
    return train_set, test_set, val_set, labelNames

def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}

def train_model_ss_subject(sub_id,target_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ss_subject(time_length,sub_id)
    print("Dataset loading completes.")
    
    best_val_acc = 0
    best_val_path = target_dir
    


    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')
            if val_acc_1>best_val_acc:
                best_val_acc = val_acc_1
                best_val_path = checkpoint_path+'-'+str(int(global_step_value))
    return best_val_acc,best_val_path


def test_model_ss_subject(sub_id,model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ss_subject(time_length,sub_id)
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats


In [3]:
best_val_rnn = {}

sub_id = 9
num_cls = 4
mstp = 5000
lfrq = 10
bsz = 50
msf = 100
tr = './trained_model_final/DCNN_RNN'
if not os.path.exists(tr):
    os.mkdir(tr)
start = 0
stop = 250
step = 10
time_length = 700
time_bin = 350
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True
acc = 'Accuracy_rnn'
path = 'Path_rnn'

best_val_rnn[acc],best_val_rnn[path] = train_model_ss_subject(sub_id,tr,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-17 16:39:15.977533: step 0, examples 0, loss = 1.418847442 (24.822 examples/sec; 2.014 sec/batch)
Top 1 validation accuracy: 0.23349057137966156 and top 2 validation accuracy: 0.5094339847564697
Model Saved!
2019-03-17 16:39:36.293556: step 10, examples 500, loss = 1.409201741 (33.740 examples/sec; 1.482 sec/batch)
2019-03-17 16:39:51.847917: step 20, examples 1000, loss = 1.401433945 (30.518 examples/sec; 1.638 sec/batch)
2019-03-17 16:40:07.143590: step 30, examples 1500, loss = 1.395565629 (33.580 examples/sec; 1.489 sec/batch)
2019-03-17 16:40:22.439263: step 40, examples 2000, loss = 1.405897260 (30.993 examples/sec; 1.613 sec/batch)
2019-03-17 16:40:37.713879: step 50, examples 2500, loss = 1.393693447 (32.807 examples/sec; 1.524 sec/batch)
2019-03-17 16:40:52.804006: step 60, examples 3000, loss = 1.382494569 (33.717 examples/sec; 1.483 sec/batch)
2019-03-17 16:41:08.102687: step 70, examples 3500, loss = 1.375700831 (33.025

2019-03-17 16:57:04.512830: step 670, examples 33500, loss = 1.266569495 (30.744 examples/sec; 1.626 sec/batch)
2019-03-17 16:57:21.017719: step 680, examples 34000, loss = 1.220920801 (31.225 examples/sec; 1.601 sec/batch)
2019-03-17 16:57:37.620868: step 690, examples 34500, loss = 1.232403636 (31.109 examples/sec; 1.607 sec/batch)
2019-03-17 16:57:53.789864: step 700, examples 35000, loss = 1.289508462 (32.829 examples/sec; 1.523 sec/batch)
Top 1 validation accuracy: 0.44811320304870605 and top 2 validation accuracy: 0.698113203048706
Model Saved!
2019-03-17 16:58:14.921057: step 710, examples 35500, loss = 1.095586896 (28.301 examples/sec; 1.767 sec/batch)
2019-03-17 16:58:31.320662: step 720, examples 36000, loss = 1.177868605 (31.402 examples/sec; 1.592 sec/batch)
2019-03-17 16:58:47.134713: step 730, examples 36500, loss = 0.987768590 (33.946 examples/sec; 1.473 sec/batch)
2019-03-17 16:59:02.750237: step 740, examples 37000, loss = 1.063945770 (31.363 examples/sec; 1.594 sec/ba

2019-03-17 17:15:16.992128: step 1340, examples 67000, loss = 0.985413969 (32.193 examples/sec; 1.553 sec/batch)
2019-03-17 17:15:32.557519: step 1350, examples 67500, loss = 0.968541205 (32.402 examples/sec; 1.543 sec/batch)
2019-03-17 17:15:48.049714: step 1360, examples 68000, loss = 1.053649902 (32.152 examples/sec; 1.555 sec/batch)
2019-03-17 17:16:03.525866: step 1370, examples 68500, loss = 0.766393304 (31.206 examples/sec; 1.602 sec/batch)
2019-03-17 17:16:18.947876: step 1380, examples 69000, loss = 1.008464217 (32.069 examples/sec; 1.559 sec/batch)
2019-03-17 17:16:34.206451: step 1390, examples 69500, loss = 0.922006249 (30.370 examples/sec; 1.646 sec/batch)
2019-03-17 17:16:49.728726: step 1400, examples 70000, loss = 1.195052624 (31.482 examples/sec; 1.588 sec/batch)
Top 1 validation accuracy: 0.5 and top 2 validation accuracy: 0.7547169923782349
Model Saved!
2019-03-17 17:17:10.784716: step 1410, examples 70500, loss = 1.001855135 (32.110 examples/sec; 1.557 sec/batch)
20

Top 1 validation accuracy: 0.5825471878051758 and top 2 validation accuracy: 0.801886796951294
Model Saved!
2019-03-17 17:33:15.607538: step 2010, examples 100500, loss = 0.835287333 (32.360 examples/sec; 1.545 sec/batch)
2019-03-17 17:33:31.235093: step 2020, examples 101000, loss = 0.960037410 (32.318 examples/sec; 1.547 sec/batch)
2019-03-17 17:33:46.871673: step 2030, examples 101500, loss = 0.787534177 (31.442 examples/sec; 1.590 sec/batch)
2019-03-17 17:34:02.396956: step 2040, examples 102000, loss = 0.726289213 (32.089 examples/sec; 1.558 sec/batch)
2019-03-17 17:34:18.056597: step 2050, examples 102500, loss = 0.820033073 (31.225 examples/sec; 1.601 sec/batch)
2019-03-17 17:34:33.702201: step 2060, examples 103000, loss = 0.963117063 (30.782 examples/sec; 1.624 sec/batch)
2019-03-17 17:34:49.220465: step 2070, examples 103500, loss = 0.768521369 (32.508 examples/sec; 1.538 sec/batch)
2019-03-17 17:35:04.671551: step 2080, examples 104000, loss = 0.867734134 (32.402 examples/se

2019-03-17 17:51:00.588927: step 2670, examples 133500, loss = 0.892311871 (31.966 examples/sec; 1.564 sec/batch)
2019-03-17 17:51:16.062073: step 2680, examples 134000, loss = 0.622710824 (31.946 examples/sec; 1.565 sec/batch)
2019-03-17 17:51:31.969092: step 2690, examples 134500, loss = 0.688604951 (30.809 examples/sec; 1.623 sec/batch)
2019-03-17 17:51:47.554536: step 2700, examples 135000, loss = 0.841308177 (32.172 examples/sec; 1.554 sec/batch)
Top 1 validation accuracy: 0.6438679099082947 and top 2 validation accuracy: 0.823113203048706
Model Saved!
2019-03-17 17:52:08.441076: step 2710, examples 135500, loss = 0.744677126 (33.468 examples/sec; 1.494 sec/batch)
2019-03-17 17:52:23.939287: step 2720, examples 136000, loss = 0.801290929 (32.256 examples/sec; 1.550 sec/batch)
2019-03-17 17:52:39.595920: step 2730, examples 136500, loss = 0.767263412 (32.276 examples/sec; 1.549 sec/batch)
2019-03-17 17:52:55.163315: step 2740, examples 137000, loss = 0.792334378 (32.466 examples/se

2019-03-17 18:08:43.699081: step 3330, examples 166500, loss = 0.889246166 (32.381 examples/sec; 1.544 sec/batch)
2019-03-17 18:08:59.169219: step 3340, examples 167000, loss = 0.681951821 (31.884 examples/sec; 1.568 sec/batch)
2019-03-17 18:09:14.749650: step 3350, examples 167500, loss = 0.715931654 (33.003 examples/sec; 1.515 sec/batch)
2019-03-17 18:09:30.222794: step 3360, examples 168000, loss = 0.670045555 (32.007 examples/sec; 1.562 sec/batch)
2019-03-17 18:09:45.705965: step 3370, examples 168500, loss = 0.656990647 (33.468 examples/sec; 1.494 sec/batch)
2019-03-17 18:10:01.229243: step 3380, examples 169000, loss = 0.696973383 (34.109 examples/sec; 1.466 sec/batch)
2019-03-17 18:10:16.253193: step 3390, examples 169500, loss = 0.780916810 (34.179 examples/sec; 1.463 sec/batch)
2019-03-17 18:10:31.489709: step 3400, examples 170000, loss = 0.551340163 (31.363 examples/sec; 1.594 sec/batch)
Top 1 validation accuracy: 0.6320754885673523 and top 2 validation accuracy: 0.839622616

2019-03-17 18:26:34.410230: step 4000, examples 200000, loss = 0.703410745 (32.850 examples/sec; 1.522 sec/batch)
Top 1 validation accuracy: 0.6580188870429993 and top 2 validation accuracy: 0.8561320900917053
Model Saved!
2019-03-17 18:26:55.314824: step 4010, examples 200500, loss = 0.670727849 (31.925 examples/sec; 1.566 sec/batch)
2019-03-17 18:27:10.810026: step 4020, examples 201000, loss = 0.666491210 (32.916 examples/sec; 1.519 sec/batch)
2019-03-17 18:27:26.388452: step 4030, examples 201500, loss = 0.631522775 (32.381 examples/sec; 1.544 sec/batch)
2019-03-17 18:27:41.922760: step 4040, examples 202000, loss = 0.672763467 (31.722 examples/sec; 1.576 sec/batch)
2019-03-17 18:27:57.592426: step 4050, examples 202500, loss = 0.554918110 (31.702 examples/sec; 1.577 sec/batch)
2019-03-17 18:28:13.125732: step 4060, examples 203000, loss = 0.639123261 (33.245 examples/sec; 1.504 sec/batch)
2019-03-17 18:28:28.633970: step 4070, examples 203500, loss = 0.580334842 (32.423 examples/s

2019-03-17 18:44:37.294314: step 4660, examples 233000, loss = 0.597392738 (32.635 examples/sec; 1.532 sec/batch)
2019-03-17 18:44:52.733985: step 4670, examples 233500, loss = 0.593674898 (33.411 examples/sec; 1.496 sec/batch)
2019-03-17 18:45:08.030727: step 4680, examples 234000, loss = 0.553516805 (32.536 examples/sec; 1.537 sec/batch)
2019-03-17 18:45:23.197591: step 4690, examples 234500, loss = 0.563838303 (31.374 examples/sec; 1.594 sec/batch)
2019-03-17 18:45:38.404740: step 4700, examples 235000, loss = 0.646329105 (31.500 examples/sec; 1.587 sec/batch)
Top 1 validation accuracy: 0.6603773832321167 and top 2 validation accuracy: 0.8349056839942932
Model Saved!
2019-03-17 18:45:59.052268: step 4710, examples 235500, loss = 0.593142688 (33.382 examples/sec; 1.498 sec/batch)
2019-03-17 18:46:14.130860: step 4720, examples 236000, loss = 0.657857180 (33.329 examples/sec; 1.500 sec/batch)
2019-03-17 18:46:29.278029: step 4730, examples 236500, loss = 0.678813875 (32.650 examples/s

In [4]:
# Test model accuracy on each subject
subject_id = np.arange(10)
start = 0
stop = 250
step = 10
time_length = 700
time_bin = 350
#Nb = BATCH_SIZE
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True
path = 'Path_rnn' 
model_dir = best_val_rnn[path]

for sub_id in subject_id:
    if sub_id < 9:
        print('Test Accuracy for subject {}:'.format(sub_id))
    else:
        print('Test Accuracy for all subjects:')
    top_1_acc_test, top_2_acc_test = test_model_ss_subject(sub_id,model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

Test Accuracy for subject 0:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_RNN\model.ckpt-4801
[test  step 4801] loss: 1.39975; top_1_accuracy: 0.52000; top_5_accuracy: 0.820000 (0.976 sec/batch; 102.414 instances/sec)
top_1_accuracy_test =  0.52 top_2_accuracy_test =  0.82
Test Accuracy for subject 1:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_RNN\model.ckpt-4801
[test  step 4801] loss: 1.74371; top_1_accuracy: 0.50000; top_5_accuracy: 0.770000 (0.977 sec/batch; 102.332 instances/sec)
top_1_accuracy_test =  0.5 top_2_accuracy_test =  0.77
Test Accuracy for subject 2:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_RNN\model.ckpt-4801
[test  step 4801] loss: 1.15001; top_1_accuracy: 0.73000; top_5_accuracy: 0.880000 (0.998 sec/batch; 100.238 instances/sec)
top_1_accuracy_test = 